# References and Resources
- This tuto is inspired by : https://github.com/azinonos/MNIST_DL_Tutorial

- Neural Network structure Visualization : https://alexlenail.me/NN-SVG/index.html

- All cell outputs in orange color

# Import necessary Deep Learning librairies (keras, sklearn...) and set initial parameters
- Number of layers and depth can be adapted (default : 2 x 128 neuron layers)

- batch_size and epochs can be also set with variables here

In [ ]:
#########################################################################################################################################
# Computer Vision Demo Jupyter Notebook based on MNSIT dataset
# Last Updated: 2024 08 27
#########################################################################################################################################
#
# Use a python3 virtualenv with Python >= 3.10.x (venv or anaconda)
#
# With conda use :
#    % conda create --name computer-vision-demo python=3.10
#    % python --version
#      Python 3.10.14
#    % conda env list
#    % conda activate computer-vision-demo
#    % conda install --file requirements.txt (or pip3 install -r requirements.txt)
#
# Recommandations : use numpy<2.x with Tensorflow to avoid conflicts

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Install keras in anaconda : https://oindrilasen.com/2021/02/how-to-install-and-import-keras-in-anaconda-jupyter-notebooks/
#   % conda install anaconda::keras
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from keras.backend import clear_session
from sklearn.metrics import confusion_matrix
import seaborn as sns
from IPython.display import display, HTML
import importlib

%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress tensorflow warnings

# Debug : Measure time to import modules
# modules = ['os', 'time', 'numpy', 'pandas', 'matplotlib.pyplot', 'keras', 'sklearn.metrics', 'seaborn', 'IPython.display']
# for module in modules:
#    start_time = time.time()
#    importlib.import_module(module)
#   end_time = time.time()
#    print(f"Importing {module} took {end_time - start_time:.2f} seconds")

# Package used to plot the training phase
# See also : https://medium.com/inbrowserai/simple-diagrams-of-convoluted-neural-networks-39c097d2925b
from livelossplot import PlotLossesKeras

# Hyper-parameters that define the number and depth of hidden layers
_hidden_layer_number=2       # Number of layers in the Neural Network (min=2)
_hidden_layer_depth=128      # Depth of each layer = number of Neurons
_training_batch_size=1024    # Define the batch size for training (number of samples to process in parallel)
_training_epochs=10          # Define the number of epochs for training (iterations over the entire dataset)
_random_seed = 7             # Default seed to initialze the numpy random number generator (rng)
_output_cell_color='#FF5E26' # Official Pure Storage Orange for outputs

# A crucial step in ensuring reproducibility is the Numpy Librairie initialization !
#
# np.random.seed(0) sets the seed for the NumPy random number generator to a fixed value (in this case, 0). 
#
# This ensures that the subsequent random number generations will be deterministic and reproducible.
# When you set a seed for the random number generator, it means that the sequence of random numbers generated 
# by the algorithm will always start with the same value. 
# This is useful when:
#    - You want to reproduce results: By setting the seed, you can guarantee that your code will produce the 
#      same output every time, which is essential for reproducing experimental results or sharing code with others.
#    - You need consistent testing: Randomness can be difficult to test, but by fixing the seed, you can create a 
#      controlled environment for testing and validation.
# In this specific case, setting the seed might not have a direct impact on your MNIST dataset analysis, 
# but it's still a good practice to ensure reproducibility and consistency in your code.

### DEPRECATED NUMPY RANDOM NUMBER GENERATOR CODE => Old style
# start_time = time.time()
# np.random.seed(0)
# elapsed_time = (time.time() - start_time)
# display(HTML(f"<h1 style='color:{_output_cell_color};'>Neural Network has been initialized with random seed: \
#         {_random_seed} in {elapsed_time:.2f} seconds</h1>"))

### NEW NUMPY RANDOM NUMBER GENERATOR CODE => New recommended Random Generator
start_time = time.time()

# Create a SeedSequence instance using our initial _random_seed
seed_sequence = np.random.SeedSequence(_random_seed)

# Create a BitGenerator instance using the SeedSequence
bit_generator = np.random.PCG64(seed_sequence)

# Create a Generator instance (: rng will be used in our code to generate random numbers)
rng = np.random.Generator(bit_generator)

# Use the Generator instance for random number generation (test example)
#    random_numbers = rng.random(10)  # Generates 10 random numbers
#    print(random_numbers)

elapsed_time = (time.time() - start_time)

display(HTML(f"<h1 style='color:{_output_cell_color};'>Neural Network has been initialized with random seed: \
             {_random_seed} in {elapsed_time:.2f} seconds</h1>"))

# Split the Dataset to prepare for training and testing phases

In [ ]:
# Import the MNIST Dataset directly from keras included datasets
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Print the shape of the training and tests sets (images + labels)
display(HTML(f"<h1 style='color:{_output_cell_color};'>The \'x_train\' structure shape (images) is {x_train.shape}</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>The \'y_train\' structure shape (labels) is {y_train.shape}</h1>"))

# Visualize all the possible output classes (or modalities) images from the MNIST Dataset

In [ ]:
# Define the number of classes (e.g., digits 0-9)
num_classes = 10

# Create a figure with subplots for each class
f, ax = plt.subplots(1, num_classes, figsize=(20,20))

display(HTML(f"<h1 style='color:{_output_cell_color};'>Display images of all possible \'modalities\' in \
             the MNIST dataset (total of 10 modalities): </h1>"))

# Loop on the x_train dataset to display all possible modalities in the dataset (test on y_test that we know is true)
for i in range(0, num_classes):
  # Get the first sample from the current class (assuming x_train and y_train are available)
  sample = x_train[y_train == i][0]

  # Display the sample as an image using grayscale colormap
  ax[i].imshow(sample, cmap='gray')

  # Set the title of the subplot with the label number
  ax[i].set_title("Label: {}".format(i), fontsize=16)

  # Remove the axis for better visualization
  ax[i].axis('off')

# Show the plot
plt.show()

# Visualize a random sample image from the training set in its numeric matrix form

In [ ]:
# Pick a random index and print the [28x28] matrix corresponding to the image at this index
# We will set 0 and 1 values only in the matrix to show the numbers with '1'
random_idx = np.random.choice(len(x_train))

# Print image matrix with values normalized to 0 ('-') or 1 ('x')
image = x_train[random_idx].reshape((28, 28))

display(HTML(f"<h1 style='color:{_output_cell_color};'>Display an image at a random index \'{random_idx}\' \
             in the x_train MNIST dataset'</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>This is what the Neural Network will have as an input: \
             can you see that it is a \'{y_train[random_idx]}\' ??</h1>"))

for row in image:
  for pixel in row:
    if pixel > 127:
      print(str(pixel).zfill(3), end=' ') 
    else:
      print(str(pixel).zfill(3), end=' ') 
  print()

# Now, we will transform y_train & y_test labels from integers (0 to 9) into a 'one-hot-encoded' matrix:

<b>
    For example :

      [5 0 4 ... 1 3 2] 
    
    will be transformed in this form :

      [[[0. 0. 0. 0. 0. 1. ... 0. 0. 0.]  ===> one-hot encodeing for 5
        [1. 0. 0. 0. 0. 0. ... 0. 0. 0.]  ===> one-hot encodeing for 0
        [0. 0. 0. 0. 1. 0. ... 0. 0. 0.]  ===> one-hot encodeing for 4
        ...
        [0. 1. 0. 0. 0. 0. ... 0. 0. 0.]  ===> one-hot encodeing for 1
        [0. 0. 0. 1. 0. 0. ... 0. 0. 0.]  ===> one-hot encodeing for 3
        [0. 0. 1. 0. 0. 0. ... 0. 0. 0.]] ===> one-hot encodeing for 2
        ...

In [ ]:
# In Keras, when working with categorical labels (i.e., discrete outcomes), you need to convert them into a categorical format. 
# This is because the default loss function and optimization algorithms assume that the outputs are real-valued (continuous).

# The to_categorical() function from Keras' utils module does just that: it converts integer-encoded categorical labels into one-hot 
# encoded arrays.
#
# Here's what happens:
#    For each sample in y_train and y_test, you get a vector of length num_classes, where:
#       The index corresponding to the true class label is set to 1 (i.e., True or 1).@
#       All other indices are set to 0 (i.e., False or 0).
# This one-hot encoded representation allows you to use categorical cross-entropy loss functions, such as categorical_crossentropy, 
# which is commonly used for multi-class classification problems.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

display(HTML(f"<h1 style='color:{_output_cell_color};'>Now, \'y_train\' labels has been \'hot-encoded\' and look like below : \
             the correpsonding label is where you see a \'1\'.</h1>"))

# Loop through some samples in the training data and print its label again (after conversion)
for i in range(4):
  print(y_train[i])

display(HTML(f"<h1 style='color:{_output_cell_color};'>This is necessary to use the \'categorical cross-entropy loss function\'</h1>"))

# Prepare Data for the training phase

### Step 1 : Normalize input images in theh range [0,1]


In [ ]:
# Normalize the pixel values of the training images by dividing each value by 255.0

# This code is normalizing the pixel values of the training and test images to be between 0 and 1, 
# which is a common practice in deep learning models.
# In this case, it's likely that the original pixel values were between 0 and 255 (i.e., 8-bit grayscale), 
# so dividing each value by 255.0 brings them down to the range [0, 1]. This normalization helps to:
#   Reduce the effect of large pixel values on the model's training and testing
#   Make the model more robust to small variations in brightness or contrast
#   Improve the convergence and accuracy of the model
# Note that this is a simple form of data normalization. 
# In some cases, you may need to use more sophisticated techniques, such as mean subtraction or standardization, 
# depending on the specific characteristics of your dataset.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Pick a random image from the x_train set with a random index
# random_idx = np.random.choice(len(x_train))     # Deprecated random numpy generator 
random_idx = rng.choice(len(x_train))

image = x_train[random_idx].reshape((28, 28))

display(HTML(f"<h1 style='color:{_output_cell_color};'>We picked a random image at index \'{random_idx}\' \
             in the x_train MNIST dataset.</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>The image matrix from \'x_train\'x transformed with \
             decimal values .xxxx is displayed below.</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>The corresponding \'y_train\'x encoded label is: \' \
            {y_train[random_idx]}\'</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>===> Can you say which number is it ??</h1>"))

# Print the image matrix with values in the range [0,1]
for row in image:
    for pixel in row:
        if pixel > 127:
            print("{:.4f}".format(pixel), end=' ')  # Display pixel value with 3 decimal places
        else:
            print("{:.4f}".format(pixel), end=' ')  # Display pixel value with 3 decimal places
    print()

display(HTML(f"<h1 style='color:{_output_cell_color};'>...It is a {np.where(y_train[random_idx] ==1)[0][0]} !</h1>"))

# Prepare Data for the training phase

### Step 2 : Reshape images from  (height, width, channels) to (samples, features)

In [ ]:
# Reshape the test images from (height, width, channels) to (samples, features)

# The -1 in the reshape method is a special value that tells NumPy to automatically calculate 
# the size based on the other dimensions. 
# In this case, it means that the first dimension (the number of samples) will remain unchanged, 
# and the second dimension (the number of features) will be calculated as the product of the height, 
# width, and channels.

# Before reshaping, x_train likely had a shape like ( Samples, Height, Width, Channels ), 
# where each image is represented by a 4D array with three dimensions for spatial information 
# (height, width, and channels).

# After reshaping, x_train will have a shape like (Samples, Features), where the number of features 
# is equal to the product of height, width, and channels. 
# This is often referred to as a "flat" or "vectorized" representation, which is suitable for 
# inputting into neural networks.

# By printing the shape of x_train using print(x_train.shape), you can verify that the reshaping was 
# successful and check the new shape of your data.

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

display(HTML(f"<h1 style='color:{_output_cell_color};'>We reshaped the input \'x_train\' images to \
             be flat with the 28x28 pixels of each images.</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>\'x_train\' shape is now: {x_train.shape} so \
             we have 60000 images of 784 pixels each</h1>"))


# Create the Model : a Fully Fast Forward Connected Neural Network (CNN
- Step 1: Create the input flaten layer (768 pixels = 28 x 28)

- Step 2 : Create one or several hidden layers (adjustable depths with variable '_hidden_layer_depth')

- Step 3 : Create the output layer : here we have 10 modalities (or classes), so 10 output neurons for the classifier

- Step 4 : print the NN layout and total numbers of parameters (it can easyly reach 1 Millions parametres !!)


In [ ]:
# Clear the current Keras session
clear_session()

# Create a neural network model using the Sequential API
model = Sequential()

# Add a dense (fully-connected) layer with default of '_hidden_layer_depth'units, input shape (784,), 
# and ReLU activation. This layer will process the input data and produce an output

# INPUT LAYER

#model.add(Dense(units=_hidden_layer_depth, input_shape=(784,), activation='relu'))    # Deprecated method
model.add(Input(shape=(784,), name='input_layer'))

# HIDDEN LAYERS

for i in range(_hidden_layer_number):
    # Add another dense layer with a default of 128 units and ReLU activation
    # This layer will process the output from the previous layer and produce another output
    #model.add(Dense(units=_hidden_layer_depth, activation='relu')
    model.add(Dense(units=_hidden_layer_depth, activation='relu', name=f'hidden_layer_{i+1}'))

# DROPOUT LAYER

# Add a dropout layer with a drop rate of 0.25 (25%)
# Dropout randomly sets 25% of the neurons to zero during training to prevent overfitting
model.add(Dropout(0.25, name='dropout_layer'))

# OUTPUT LAYER

# Add a final dense layer with 10 units and softmax activation
# This layer will produce a probability distribution over the 10 classes
model.add(Dense(units=10, activation='softmax', name='output_layer'))

# Compile the model with categorical cross-entropy loss function, Adam optimizer, and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

display(HTML(f"<h1 style='color:{_output_cell_color};'>Our Neural network is composed of 1 Input layer, {_hidden_layer_number} \
             hidden layers with each {_hidden_layer_depth} neurons and one Output Layer as described in the summary</h1>"))

# Print a summary of the model architecture using the `summary()` method
model.summary()

#                     +---------------+
#                     |  Input Layer  |
#                     +---------------+
#                             |
#                             | (784,)
#                             v
# +-----------------------+       +-----------------------+
# |      Dense Layer      |       |      Dense Layer      |
# |      (128, ReLU)      |       |      (128, ReLU)      |
# +-----------------------+       +-----------------------+
#                             |
#                             | (0.25 dropout)
#                             v
# +-----------------------+       +-----------------------+
# |      Dropout Layer    |       |        None           |
# +-----------------------+       +-----------------------+
#                             |
#                             |
#                             v
# +-----------------------+       +-----------------------+
# |     Final Dense Layer |       |  Softmax Activation   |
# |      (10, Softmax)    |       |                       |
# +-----------------------+       +-----------------------+

# Note that this schema is a simplified representation of the model architecture. 
# The actual connections between layers may be more complex and involve additional 
# operations such as convolutional layers, pooling layers, etc.


# Train the model by using tmethod 'fit' and define a bath_size

In [ ]:
# Batch size for training (number of samples to process in parallel)
#    See variable : _training_batch_size

# Define the number of epochs for training (iterations over the entire dataset)
#    See variable : _training_epochs

# Full stack trace mode for debugging
# keras.config.disable_traceback_filtering()

# Start the timer
start_time = time.time()

# Train the model using the `fit` method
# x_train: input data, y_train: target labels
model.fit(x=x_train, y=y_train, 
          validation_data=(x_test, y_test), 
          callbacks=[PlotLossesKeras()],
          verbose = 0,
          batch_size=_training_batch_size,
          epochs=_training_epochs)

# End the timer
end_time = time.time()

# Elapsed time for the training phase
elapsed_time = end_time - start_time

display(HTML(f"<h1 style='color:{_output_cell_color};'>The model.fit() function to train our model \
             took {elapsed_time:.2f} seconds to run.</h1>"))


# Evaluate the model by calculationg the Accuracy

In [ ]:
# Evaluate the model on the 'test' dataset using the `evaluate` method
test_loss, test_acc = model.evaluate(x_test, y_test)

display(HTML(f"<h1 style='color:{_output_cell_color};'>Loss after training phase = {round(test_loss, 2)}</h1>"))
display(HTML(f"<h1 style='color:{_output_cell_color};'>Accuracy after training phase = {round(test_acc, 2)}</h1>"))

# Use the model to make predictions on the x_test dataset (never seen by the model during training)

In [ ]:
display(HTML(f"<h1 style='color:{_output_cell_color};'>Now we can predict all labels for the \'y_test\' testing set:</h1>"))

# Here's what each part does:
#    model.predict(x_test) : This line uses the trained model to make predictions on the test data. 
#    The output is a NumPy array containing the predicted probability distribution for each sample
#    in the test data.
#
#    y_pred_classes = np.argmax(y_pred, axis=1) : This line converts the predicted probabilities 
#    into class labels by taking the index of the maximum value along the specified axis (in this case, axis 1). 
#    The output is a NumPy array containing the predicted class labels for each sample in the test data.
#
#    print(y_pred) : This line prints out the predicted probability distributions for each sample in the test data. 
#    Each row represents a single prediction, and the columns represent the different classes (or outcomes) that the model can predict.
#
#    print(y_pred_classes) : This line prints out the predicted class labels for each sample in the test data.

# Use the model to make predictions on the test data
y_pred = model.predict(x_test)

# Convert the predicted probabilities into class labels using `argmax`
y_pred_classes = np.argmax(y_pred, axis=1)

display(HTML(f"<h1 style='color:{_output_cell_color};'>y_pred_classes table contains all predicted \
             labels (with a total of {len(y_pred_classes)} labels)</h1>"))

display(HTML(f"<h1 style='color:{_output_cell_color};'>y_pred_classes table contains all \
             predicted labels : {y_pred_classes}</h1>"))

# Dataframe with predicted labels to have a better display...
df = pd.DataFrame(y_pred_classes, columns=['Predicted Classes'])
df = df.reset_index(drop=True).reset_index().rename(columns={'index': 'y_pred Index'})

# Display the 100 fisrt predicted labels
print(df.head(20).to_string(index=False))

# Pick a random image form x_test the run the prediction and display results with 'predict' and 'true'

Note : each run of this cell will pick a different image form the x_test dataset (random index)

In [ ]:
# Single PREDICTION Example on a random sample from the x_test dataset 
# (not known during the training phase)

# Select a single random example from the test data for visualization by choosing a 
# random index within the length of x_test

#random_idx = np.random.choice(len(x_test))
random_idx = rng.choice(len(x_test))

# Get the input data (features) at the chosen index
x_sample = x_test[random_idx]
display(HTML(f"<h1 style='color:{_output_cell_color};'>Predicting Image at Index = {random_idx}, \
             Size of x_sample = {len(x_sample)}</h1>"))

# Now, extract the true and predicted class labels for this example

# And now, we have to onvert the one-hot encoded label to a class label (0-9)

# Let's explain more in depth this part:
#   np.argmax: This is a function from the NumPy library that finds the index of the maximum value 
#      along a specified axis.
#
#   y_test: This is a 2D NumPy array representing the true labels for the test data. Each row corresponds 
#      to a single sample, and each column represents a possible class label (in this case, digit classes 0-9).
#
#   axis=1: This specifies that we want to find the maximum value along the second axis (i.e., the rows). 
#     In other words, we're looking for the most likely class label for each individual sample.
#
# When you run this code, NumPy will iterate through each row of y_test, find the maximum value in that row, 
# and return the corresponding index. 
#
# => This is equivalent to converting the one-hot encoded labels into a single class label (e.g., [0, 0, 1, 0] 
# becomes 2, since 2 is the most likely class).
# The resulting array y_true will contain the true class labels for each sample in the test data.
y_true = np.argmax(y_test, axis=1)

# Get the true class label for this example
y_sample_true = y_true[random_idx]

# Get the predicted class label for this example
y_sample_pred_class = y_pred_classes[random_idx]

display(HTML(f"<h1 style='color:{_output_cell_color};'>\'Predicted Label\' versus \'True label\' in y_test</h1>"))

plt.title("Predicted: {}, True: {}".format(y_sample_pred_class, y_sample_true), fontsize=16)
plt.imshow(x_sample.reshape(28, 28), cmap='gray')

# Show the plot
plt.show()

# Calculate and display the Confusion Matrix of the model

In [ ]:
# This code calculates the confusion matrix : a table that summarizes the predictions made by a 
# classification model, using the true and predicted labels. 
# Then, it creates a plot for the confusion matrix using Seaborn's heatmap function. 
# The annot=True argument means that each cell in the heatmap will be annotated with the number 
# of correct/incorrect predictions. 
# The fmt='d' argument specifies the format of the annotations (in this case, decimal integers).

# Calculate the confusion matrix using true and predicted labels
confusion_mtx = confusion_matrix(y_true, y_pred_classes)

display(HTML(f"<h1 style='color:{_output_cell_color};'>Display the Confusion Matrix of our model \
             showing \'Predicted\' versus \'True\'</h1>"))

# Create a plot for the confusion matrix
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(confusion_mtx, annot=True, fmt='d', ax=ax, cmap="Blues")
ax.set_xlabel('Predicted Label')
ax.set_ylabel('True Label')
ax.set_title('Confusion Matrix');

# Display the plot
plt.show()

# Investigate Errors of our Model, in which case it fails do predict correctly ?

In [ ]:
# This code identifies the incorrect predictions by subtracting the true labels from 
# the predicted class labels. 
# It then extracts the relevant information (predicted classes, probabilities, true labels, 
# and input data) only for these incorrect predictions.

# The errors array is a boolean mask that indicates which indices in the original arrays correspond 
# to incorrect predictions.
# The indexing operation [errors] uses this mask to extract the corresponding elements from each array, 
# resulting in separate arrays for the errors (incorrect predictions).

# Create a boolean array indicating where predictions are incorrect
errors = (y_pred_classes - y_true != 0)

# Extract the predicted classes, predicted probabilities, true labels, and 
# input data for incorrect predictions only
y_pred_classes_errors = y_pred_classes[errors]  # Predicted class labels for errors
y_pred_errors = y_pred[errors]                  # Predicted probabilities for errors
y_true_errors = y_true[errors]                  # True class labels for errors
x_test_errors = x_test[errors]                  # Input data (features) for errors

# Calculate the maximum predicted error probability for each sample
y_pred_errors_probability = np.max(y_pred_errors, axis=1)

# Extract the true class probability errors from the predicted errors
true_probability_errors = np.diagonal(np.take(y_pred_errors, y_true_errors, axis=1))

# Compute the difference between the predicted error probability and the true class probability error
diff_errors_pred_true = y_pred_errors_probability - true_probability_errors

# Get list of indices of sorted differences
sorted_idx_diff_errors = np.argsort(diff_errors_pred_true)
top_idx_diff_errors = sorted_idx_diff_errors[-5:] # 5 last ones

# Show Top Errors
num = len(top_idx_diff_errors)
f, ax = plt.subplots(1, num, figsize=(30,30))

print()
display(HTML(f"<h1 style='color:{_output_cell_color};'>Display samples where our model is failing the most \
             (and compare with the confusion matrix above)</h1>"))

for i in range(0, num):
  idx = top_idx_diff_errors[i]
  sample = x_test_errors[idx].reshape(28,28)
  y_t = y_true_errors[idx]
  y_p = y_pred_classes_errors[idx]
  ax[i].imshow(sample, cmap='gray')
  ax[i].set_title("Predicted label :{}\nTrue label: {}".format(y_p, y_t), fontsize=22)

# Show the plot
plt.show()

# Congratulations you reached the dnd of the simple Computer Vision Demo ! 

## Summary of what you did :

### - Import the clean MNIST dataset from keras
### - Split the dataset with a training set and a testing set
### - Visualize the dataset 
### - Prepare the dataset for the training phase
### - Build a simple Neural Network model from scratch
### - Train the model
### - Evaluate the model (accuracy and confusion matrix)
### - Investigate errors of the model

# Thanks !